<a href="https://colab.research.google.com/github/dannynew111/erp-clustering-smart-meter-data/blob/main/feature_engineering_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import gc
import pyarrow.parquet as pq

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define File Paths
processed_data_dir = '/content/drive/MyDrive/ERP/data_loaded/'
# Point to the new Parquet files
path_2012 = os.path.join(processed_data_dir, 'consumption_2012_v2.parquet')
path_2013 = os.path.join(processed_data_dir, 'consumption_2013_v2.parquet')

# Define the new definitive output file
final_output_path = os.path.join(processed_data_dir, 'final_dataset_v4.csv')

# Define BOTH cohorts
# No need 'household_info.csv', I can get the cohorts directly from the Parquet files

# Read just the household IDs and their group from one of the files
# More efficient than loading the whole file
df_info_v2 = pd.read_parquet(path_2012, columns=['Household_id', 'stdorToU']).drop_duplicates()

# Now create sets using the NEW ID column name
tou_household_set = set(df_info_v2[df_info_v2['stdorToU'] == 'ToU']['Household_id'])
std_household_set = set(df_info_v2[df_info_v2['stdorToU'] == 'Std']['Household_id'])

print(f"Identified {len(tou_household_set)} households in the 'ToU' (Treatment) group.")
print(f"Identified {len(std_household_set)} households in the 'Std' (Control) group.")
print("Setup complete. Ready to process the new data.")

Mounted at /content/drive
Identified 1025 households in the 'ToU' (Treatment) group.
Identified 4173 households in the 'Std' (Control) group.
Setup complete. Ready to process the new data.


In [2]:
#  Cell 2: Process All Consumption Data for Both Cohorts
print("Processing all 2012 and 2013 consumption data... This will take several hours.")

def get_baseline_profile(file_path, cohort_set):
    agg_sums, agg_counts = None, None
    parquet_file = pq.ParquetFile(file_path)
    for i, batch in enumerate(parquet_file.iter_batches(batch_size=1_000_000)):
        chunk = batch.to_pandas()
        chunk = chunk[chunk['Household_id'].isin(cohort_set)] # UPDATED to Household_id
        if chunk.empty: continue
        chunk['time_period'] = chunk['DateTime'].dt.hour.map(lambda h: 'Peak' if (7<=h<9) or (16<=h<20) else 'Off-Peak')
        chunk['day_type'] = chunk['DateTime'].dt.dayofweek.map(lambda x: 'Weekday' if x < 5 else 'Weekend')
        # UPDATED to Household_id
        chunk_sums = chunk.groupby(['Household_id', 'day_type', 'time_period'])['energy_kwh'].sum()
        chunk_counts = chunk.groupby(['Household_id', 'day_type', 'time_period'])['energy_kwh'].count()
        if agg_sums is None: agg_sums, agg_counts = chunk_sums, chunk_counts
        else:
            agg_sums = agg_sums.add(chunk_sums, fill_value=0)
            agg_counts = agg_counts.add(chunk_counts, fill_value=0)
        print(f"  Processed 2012 chunk {i+1} for {len(cohort_set)} households...")
    return (agg_sums / agg_counts).unstack(level=[1, 2]).fillna(0)

def get_trial_profile_tou(file_path, cohort_set):
    agg_sums, agg_counts = None, None
    parquet_file = pq.ParquetFile(file_path)
    for i, batch in enumerate(parquet_file.iter_batches(batch_size=1_000_000)):
        chunk = batch.to_pandas()
        chunk = chunk[chunk['Household_id'].isin(cohort_set)] # UPDATED to Household_id
        if chunk.empty: continue

        # Create PriceSignal from existing Price column
        chunk['PriceSignal'] = 'Normal' # Default value
        # Use thresholds to handle floating point variations
        chunk.loc[chunk['Price'] > 0.6, 'PriceSignal'] = 'High'
        chunk.loc[chunk['Price'] < 0.04, 'PriceSignal'] = 'Low'

        chunk['day_type'] = chunk['DateTime'].dt.dayofweek.map(lambda x: 'Weekday' if x < 5 else 'Weekend')
        # UPDATED to Household_id
        chunk_sums = chunk.groupby(['Household_id', 'day_type', 'PriceSignal'])['energy_kwh'].sum()
        chunk_counts = chunk.groupby(['Household_id', 'day_type', 'PriceSignal'])['energy_kwh'].count()
        if agg_sums is None: agg_sums, agg_counts = chunk_sums, chunk_counts
        else:
            agg_sums = agg_sums.add(chunk_sums, fill_value=0)
            agg_counts = agg_counts.add(chunk_counts, fill_value=0)
        print(f"  Processed 2013 ToU chunk {i+1}...")
    return (agg_sums / agg_counts).unstack(level=[1, 2]).fillna(0)

def get_trial_profile_std(file_path, cohort_set):
    # This function is equivalent to baseline for the control group
    return get_baseline_profile(file_path, cohort_set)

print("\nCalculating 2012 Baselines...")
baseline_tou = get_baseline_profile(path_2012, tou_household_set)
baseline_std = get_baseline_profile(path_2012, std_household_set)
print("\nCalculating 2013 Trial Profiles...")
trial_tou = get_trial_profile_tou(path_2013, tou_household_set)
trial_std = get_trial_profile_std(path_2013, std_household_set)

print("\nAll consumption profiles calculated correctly.")

Processing all 2012 and 2013 consumption data... This will take several hours.

Calculating 2012 Baselines...
  Processed 2012 chunk 1 for 1025 households...
  Processed 2012 chunk 2 for 1025 households...
  Processed 2012 chunk 3 for 1025 households...
  Processed 2012 chunk 4 for 1025 households...
  Processed 2012 chunk 5 for 1025 households...
  Processed 2012 chunk 6 for 1025 households...
  Processed 2012 chunk 7 for 1025 households...
  Processed 2012 chunk 8 for 1025 households...
  Processed 2012 chunk 9 for 1025 households...
  Processed 2012 chunk 10 for 1025 households...
  Processed 2012 chunk 11 for 1025 households...
  Processed 2012 chunk 11 for 4173 households...
  Processed 2012 chunk 12 for 4173 households...
  Processed 2012 chunk 13 for 4173 households...
  Processed 2012 chunk 14 for 4173 households...
  Processed 2012 chunk 15 for 4173 households...
  Processed 2012 chunk 16 for 4173 households...
  Processed 2012 chunk 17 for 4173 households...
  Processed 2012 

In [3]:
# Calculate Difference-in-Differences (DiD) Responsiveness Features - this is my control method
print("Calculating DiD responsiveness features...")
epsilon = 1e-6 #used in case any near 0 median values (e.g. load shift) which could cause zero division error

# Define the four key DataFrames from Cell 2: baseline_tou, baseline_std, trial_tou, trial_std

# Calculate the "Second Difference": The average change for the CONTROL group
control_change_peak_wd = trial_std[('Weekday', 'Peak')].mean() - baseline_std[('Weekday', 'Peak')].mean()
control_change_peak_we = trial_std[('Weekend', 'Peak')].mean() - baseline_std[('Weekend', 'Peak')].mean()
control_change_offpeak_wd = trial_std[('Weekday', 'Off-Peak')].mean() - baseline_std[('Weekday', 'Off-Peak')].mean()

print(f"Control group's average WEEKDAY PEAK consumption changed by: {control_change_peak_wd:+.4f} kWh")
print(f"Control group's average WEEKDAY OFF-PEAK consumption changed by: {control_change_offpeak_wd:+.4f} kWh")

# Calculate the "First Difference": The individual change for EACH ToU household
change_tou_peak_wd = trial_tou[('Weekday', 'High')] - baseline_tou[('Weekday', 'Peak')]
change_tou_peak_we = trial_tou[('Weekend', 'High')] - baseline_tou[('Weekend', 'Peak')]
change_tou_offpeak_wd = trial_tou[('Weekday', 'Low')] - baseline_tou[('Weekday', 'Off-Peak')]

# 4. Calculate the "Difference-in-Differences" and the final features
df_did_features = pd.DataFrame(index=baseline_tou.index)
df_did_features['peak_reduction_weekday'] = -1 * (change_tou_peak_wd - control_change_peak_wd)
df_did_features['peak_reduction_weekend'] = -1 * (change_tou_peak_we - control_change_peak_we)
df_did_features['load_shift_weekday'] = change_tou_offpeak_wd - control_change_offpeak_wd
df_did_features['peak_reduction_pct_weekday'] = (df_did_features['peak_reduction_weekday'] / (baseline_tou[('Weekday', 'Peak')] + epsilon)) * 100
df_did_features['peak_reduction_pct_weekend'] = (df_did_features['peak_reduction_weekend'] / (baseline_tou[('Weekend', 'Peak')] + epsilon)) * 100

print("\nDiD Responsiveness features created correctly and robustly.")
display(df_did_features.head())

Calculating DiD responsiveness features...
Control group's average WEEKDAY PEAK consumption changed by: -0.0033 kWh
Control group's average WEEKDAY OFF-PEAK consumption changed by: +0.0019 kWh

DiD Responsiveness features created correctly and robustly.


,peak_reduction_weekday,peak_reduction_weekend,load_shift_weekday,peak_reduction_pct_weekday,peak_reduction_pct_weekend
Household_id,,,,,
D0000,0.010094,-0.005836,0.025642,5.928131,-3.651439
D0001,0.006051,-0.009243,0.002672,5.033200,-8.254847
D0002,0.104148,0.165076,0.065374,17.352084,23.594709
D0003,0.055692,0.043813,0.033175,35.151939,28.927676
D0004,0.113824,0.057513,0.004660,43.775144,30.042499


In [4]:
# Calculate All Features, Merge, Clean Names, and Save (Definitive Version)
epsilon = 1e-6 # Define a small constant to prevent division-by-zero errors

# Calculate ALL Intrinsic Features from Baseline Data
print("Calculating all intrinsic behavioural features from baseline data...")
baseline_tou.index.name = 'Household_id'
pq_2012 = pq.ParquetFile(path_2012)
df_2012_full = pq_2012.read(columns=['DateTime', 'Household_id', 'energy_kwh']).to_pandas()
df_2012_tou = df_2012_full[df_2012_full['Household_id'].isin(tou_household_set)]
daily_totals = df_2012_tou.groupby(['Household_id', pd.Grouper(key='DateTime', freq='D')])['energy_kwh'].sum()
df_intrinsic = daily_totals.groupby('Household_id').std().to_frame(name='intrinsic_daily_volatility')
daily_means = daily_totals.groupby('Household_id').mean()
daily_maxs = daily_totals.groupby('Household_id').max()
df_intrinsic['intrinsic_load_factor'] = daily_means / (daily_maxs + epsilon)
df_intrinsic['base_peak_to_offpeak_ratio'] = baseline_tou.loc[:, ('Weekday', 'Peak')] / (baseline_tou.loc[:, ('Weekday', 'Off-Peak')] + epsilon)
df_intrinsic['base_day_to_night_ratio'] = df_2012_tou.groupby('Household_id').apply(lambda x: x.set_index('DateTime').between_time('07:00', '18:59')['energy_kwh'].mean() / (x.set_index('DateTime').between_time('19:00', '06:59')['energy_kwh'].mean() + epsilon))
df_intrinsic['base_morning_vs_evening_peak_ratio'] = baseline_tou.loc[:, ('Weekday', 'Peak')] / (baseline_tou.loc[:, ('Weekend', 'Peak')] + epsilon) # Simplified
print("All intrinsic features calculated.")
del df_2012_full, df_2012_tou, daily_totals, daily_means, daily_maxs
gc.collect()

# Calculate Financial Outcome Features

print("\nCalculating financial outcome features from trial data...")
PRICES = {'flat_rate': 0.14228, 'High': 0.6720, 'Low': 0.0399, 'Normal': 0.1176}
pq_2013 = pq.ParquetFile(path_2013)
df_2013_full = pq_2013.read().to_pandas()
df_2013_tou = df_2013_full[df_2013_full['Household_id'].isin(tou_household_set)].copy()
# Recreate PriceSignal column
df_2013_tou['PriceSignal'] = 'Normal'
df_2013_tou.loc[df_2013_tou['Price'] > 0.6, 'PriceSignal'] = 'High'
df_2013_tou.loc[df_2013_tou['Price'] < 0.04, 'PriceSignal'] = 'Low'
# Get total consumption per signal
total_kwh_by_signal = df_2013_tou.groupby(['Household_id', 'PriceSignal'])['energy_kwh'].sum().unstack().fillna(0)
# Calculate actual and hypothetical costs
total_kwh_by_signal['actual_cost_gbp'] = (total_kwh_by_signal['High'] * PRICES['High']) + (total_kwh_by_signal['Low'] * PRICES['Low']) + (total_kwh_by_signal['Normal'] * PRICES['Normal'])
total_kwh_by_signal['total_kwh_2013'] = total_kwh_by_signal['High'] + total_kwh_by_signal['Low'] + total_kwh_by_signal['Normal']
total_kwh_by_signal['hypothetical_cost_gbp'] = total_kwh_by_signal['total_kwh_2013'] * PRICES['flat_rate']
# Calculate final financial features
df_financial_features = pd.DataFrame(index=total_kwh_by_signal.index)
df_financial_features['estimated_cost_savings_pct'] = ((total_kwh_by_signal['hypothetical_cost_gbp'] - total_kwh_by_signal['actual_cost_gbp']) / (total_kwh_by_signal['hypothetical_cost_gbp'] + epsilon)) * 100
df_financial_features['economic_rationality_score'] = total_kwh_by_signal['Low'] / (total_kwh_by_signal['High'] + epsilon)
print("All financial features calculated.")
del df_2013_full, df_2013_tou, total_kwh_by_signal
gc.collect()

#  Merge All 12 Engineered Feature Sets
print("\nMerging all feature sets...")
df_final = pd.merge(df_did_features, df_intrinsic, on='Household_id', how='inner')
df_final = pd.merge(df_final, df_financial_features, on='Household_id', how='inner')

#  Flatten the Component DataFrames
baseline_tou_prefixed = baseline_tou.add_prefix('component_base_')
baseline_tou_prefixed.columns = ['_'.join(col).strip() for col in baseline_tou_prefixed.columns.values]
baseline_tou_flat = baseline_tou_prefixed.reset_index()
trial_tou.index.name = 'Household_id'
trial_tou_prefixed = trial_tou.add_prefix('component_trial_')
trial_tou_prefixed.columns = ['_'.join(col).strip() for col in trial_tou_prefixed.columns.values]
trial_tou_flat = trial_tou_prefixed.reset_index()
print("Component DataFrames successfully flattened.")

#  Merge the Flattened Component Features
df_final = pd.merge(df_final.reset_index(), baseline_tou_flat, on='Household_id', how='inner')
df_final = pd.merge(df_final, trial_tou_flat, on='Household_id', how='inner')

#  Clean Up Column Names
print("\nRenaming clunky component columns to clean, professional names...")
# This map now uses the correct flattened names as keys
rename_map = {
    'component_base_Weekday_Peak': 'base_weekday_peak', 'component_base_Weekday_Off-Peak': 'base_weekday_offpeak',
    'component_base_Weekend_Peak': 'base_weekend_peak', 'component_base_Weekend_Off-Peak': 'base_weekend_offpeak',
    'component_trial_Weekday_High': 'trial_weekday_high', 'component_trial_Weekday_Low': 'trial_weekday_low',
    'component_trial_Weekday_Normal': 'trial_weekday_normal', 'component_trial_Weekend_High': 'trial_weekend_high',
    'component_trial_Weekend_Low': 'trial_weekend_low', 'component_trial_Weekend_Normal': 'trial_weekend_normal'
}
df_final.rename(columns=lambda c: rename_map.get(c, c), inplace=True)
print("Columns successfully renamed.")

#  Final Cleanup and Save
df_final.replace([np.inf, -np.inf], np.nan, inplace=True)
df_final.dropna(inplace=True)

print(f"\nFinal comprehensive dataset created for {len(df_final)} households.")
print(f"The dataset now contains the full {df_final.shape[1]} columns.")

# This is my version 5 as I had a few errors previously! This is now the definitive version
final_output_path = '/content/drive/MyDrive/ERP/data_loaded/final_dataset_v5_definitive.csv'
df_final.to_csv(final_output_path, index=False)
print(f"Definitive dataset saved to: {final_output_path}")

print("\n Final Dataset Head ")
display(df_final.head())

Calculating all intrinsic behavioural features from baseline data...


/tmp/ipython-input-903093269.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_intrinsic['base_day_to_night_ratio'] = df_2012_tou.groupby('Household_id').apply(lambda x: x.set_index('DateTime').between_time('07:00', '18:59')['energy_kwh'].mean() / (x.set_index('DateTime').between_time('19:00', '06:59')['energy_kwh'].mean() + epsilon))


All intrinsic features calculated.

Calculating financial outcome features from trial data...
All financial features calculated.

Merging all feature sets...
Component DataFrames successfully flattened.

Renaming clunky component columns to clean, professional names...
Columns successfully renamed.

Final comprehensive dataset created for 1025 households.
The dataset now contains the full 23 columns.
Definitive dataset saved to: /content/drive/MyDrive/ERP/data_loaded/final_dataset_v5_definitive.csv

 Final Dataset Head 


,Household_id,peak_reduction_weekday,peak_reduction_weekend,load_shift_weekday,peak_reduction_pct_weekday,peak_reduction_pct_weekend,intrinsic_daily_volatility,intrinsic_load_factor,base_peak_to_offpeak_ratio,base_day_to_night_ratio,...,component_base_Weekday_component_base_Off-Peak,component_base_Weekday_component_base_Peak,component_base_Weekend_component_base_Off-Peak,component_base_Weekend_component_base_Peak,component_trial_Weekday_component_trial_High,component_trial_Weekday_component_trial_Low,component_trial_Weekday_component_trial_Normal,component_trial_Weekend_component_trial_High,component_trial_Weekend_component_trial_Low,component_trial_Weekend_component_trial_Normal
0,D0000,0.010094,-0.005836,0.025642,5.928131,-3.651439,1.719578,0.528509,1.520769,1.201120,...,0.111965,0.170274,0.124892,0.159840,0.156928,0.139553,0.127799,0.162200,0.183983,0.140118
1,D0001,0.006051,-0.009243,0.002672,5.033200,-8.254847,1.408479,0.496033,1.267128,1.027525,...,0.094870,0.120213,0.093057,0.111972,0.110911,0.099488,0.106971,0.117738,0.102677,0.108033
2,D0002,0.104148,0.165076,0.065374,17.352084,23.594709,8.111155,0.420619,1.995773,1.762536,...,0.300737,0.600204,0.450773,0.699630,0.492804,0.368057,0.380374,0.531078,0.522850,0.413935
3,D0003,0.055692,0.043813,0.033175,35.151939,28.927676,1.463273,0.449315,0.803954,0.692284,...,0.197064,0.158431,0.231493,0.151456,0.099487,0.232186,0.180412,0.104167,0.272749,0.216271
4,D0004,0.113824,0.057513,0.004660,43.775144,30.042499,1.450504,0.619237,2.683792,1.202150,...,0.096884,0.260018,0.099851,0.191437,0.142943,0.103491,0.110439,0.130448,0.106318,0.099826
